### ACO (Ant Colony Optimization) algorithm for Smart Grids

Intro

In [81]:
interval_num = 24
modes_num = 3
num_nodes = interval_num*modes_num

In [82]:
import networkx as nx
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import random
from statistics import mean

In [83]:
np.seterr(divide='ignore', invalid='ignore')
None

### Generate random X and Y coordinates

In [84]:
xs = []
ys = []
pheromone = []
start_interval = -1
for _ in range(interval_num):
    start_interval += 1
    start_mode = 0
    for _ in range(modes_num):
        xs.append(start_interval)
        ys.append(start_mode)
        pheromone.append(1)
        start_mode += 1
print(xs)
print(ys)
schedule_map = list(zip(xs, ys, pheromone))

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23]
[0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]


### Generate the distances between all nodes

### ACO parameters

In [85]:
max_iterations = 100
num_ants = 20

rho = 0.05 # evaporation rate .. this determines how much pheromones are lost at each iteration
alpha = 1 # pheromone exponential parameter
beta = 1 # desirability exponential parameter

### Feasibility Function

In [86]:
cost_table = [300, 0, 0] # Mode 0 is fossil fuel, Mode 1 is battery, Mode 2 is Solar
generation_table = [15, 10, 5]
battery_capacity = 100 # kw/interval
battery_start = 100 # Is battery charged on schedule start?
unit_comsumption = 2
when_is_solar_available = [5-17]

def is_schedule_viable (actual_schedule):
    battery = battery_capacity
    actual_interval = 0
    for intervals in actual_schedule:
        print(intervals)
        if generation_table[intervals[0]] < unit_comsumption:
            return False
        if intervals == 2 and intervals not in when_is_solar_available:
            return False
        else:
            return True
    actual_interval += 1


### ACO functions

In [87]:
def create_colony(num_ants):
    current_schedule_all = []
    for i in range(num_ants):
        current_schedule = []
        
        actual_interval = 0
        impossible_modes = []
        while actual_interval < interval_num:
            print(f'{actual_interval} intervals')
            possible_schedules = []
            possible_schedules_weight = []
            #print(schedule_map)
            for modes in schedule_map:
                #print(modes)
                if modes[0] == actual_interval and modes[1] not in impossible_modes:
                    #print(modes[1])
                    possible_schedules.append(modes[1])
                    #print(possible_schedules)
                    possible_schedules_weight.append(modes[2])
            #print(possible_schedules)
            #print(possible_schedules_weight)
            chosen_schedule = random.choices(possible_schedules, possible_schedules_weight)
            trial_schedule = current_schedule.copy()
            trial_schedule.append(chosen_schedule)
            if is_schedule_viable(trial_schedule):
                chosen_schedule = trial_schedule
                impossible_modes = []
                actual_interval += 1
            else:
                impossible_modes.append(chosen_schedule)
            #print(chosen_schedule)
            current_schedule.append(chosen_schedule[0])
            print(is_schedule_viable(chosen_schedule))
            #print(possible_schedules)
            #print(current_schedule)
        
        current_schedule_all.append(current_schedule)
    
    return current_schedule_all
 

### Main loop of ACO

In [88]:
shortest_tour_length = float('inf')
all_solutions = np.empty((0, 2))

for i in range(0, max_iterations):
    # create the tours for all ants
    tours = create_colony(num_ants)

    # calculate the tour length for each tour
    tour_lengths = []
    for schedule in tours:
        total_cost = 0
        for interval_mode in schedule:
            print(interval_mode)
            total_cost += cost_table[interval_mode[0]]
        print(total_cost)
        if total_cost <= shortest_tour_length:
            shortest_tour_length = total_cost
            best_tour = schedule
        tour_lengths.append(total_cost)
        tour_and_distance = list(zip((tours, tour_lengths)))
    #print(tour_and_distance)
    print(f'shortest tour {best_tour}')
    print(f'length {shortest_tour_length}')
        
    # update the pheromone matrix and evaporate
    interval_start = 0
    for interval in best_tour:
        for intervals in schedule_map:
            if intervals[0] == interval_start and intervals[1] == interval:
                intervals = (intervals[0], intervals[1], intervals[2]+alpha) 
            intervals = (intervals[0], intervals[1], intervals[2]*(1-rho))
        interval_start += 1


0 intervals
[2]
[2]
True
1 intervals
[2]
[2]
True
2 intervals
[2]
[2]
True
3 intervals
[2]
[2]
True
4 intervals
[2]
[2]
True
5 intervals
[2]
[2]
True
6 intervals
[2]
[2]
True
7 intervals
[2]
[2]
True
8 intervals
[2]
[2]
True
9 intervals
[2]
[2]
True
10 intervals
[2]
[2]
True
11 intervals
[2]
[2]
True
12 intervals
[2]
[2]
True
13 intervals
[2]
[2]
True
14 intervals
[2]
[2]
True
15 intervals
[2]
[2]
True
16 intervals
[2]
[2]
True
17 intervals
[2]
[2]
True
18 intervals
[2]
[2]
True
19 intervals
[2]
[2]
True
20 intervals
[2]
[2]
True
21 intervals
[2]
[2]
True
22 intervals
[2]
[2]
True
23 intervals
[2]
[2]
True
0 intervals
[0]
[0]
True
1 intervals
[0]
[0]
True
2 intervals
[0]
[0]
True
3 intervals
[0]
[0]
True
4 intervals
[0]
[0]
True
5 intervals
[0]
[0]
True
6 intervals
[0]
[0]
True
7 intervals
[0]
[0]
True
8 intervals
[0]
[0]
True
9 intervals
[0]
[0]
True
10 intervals
[0]
[0]
True
11 intervals
[0]
[0]
True
12 intervals
[0]
[0]
True
13 intervals
[0]
[0]
True
14 intervals
[0]
[0]
True
15 int

### Shortest Path Length